### Tools

**Models** can request to call tools that perform tasks such as fetching data from a database, searching the web, or running code.
**Tools** are pairing of:

1. A Schema including the name of the tool, a description, and/or argument definations(often a JSON schema).
2. A function or coroutine to execute.

In [17]:
import os
from langchain.chat_models import init_chat_model
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [26]:
# GPT MODEL
# model = init_chat_model("gpt-4o-mini")

# GROQ MODEL
# model = init_chat_model("groq:llama-3.3-70b-versatile")

# GOOGLE MODEL
model = init_chat_model(model="google_genai:gemini-flash-latest")

response = model.stream("How airplanes fly?")


for chunk in response:
  print(chunk.text, end=" ", flush=True)

That  is a fantastic question! The flight of an airplane is a beautiful combination of physics, engineering , and the clever manipulation of air.

Here is a breakdown of the key concepts and forces that make airplanes fly:

---

## 1. The Four Forces of Flight

For an airplane to fly steadily, four fundamental forces must be in  balance:

| Force | Direction | Counteracted By | Role in Flight |
| :--- | :--- | :--- | :--- |
| **Lift** | Upward (perpendicular to the wing) | Weight | The  force that raises the aircraft and keeps it airborne. |
| **Weight** | Downward (due to gravity) | Lift | The total mass of the aircraft pulling it toward Earth. |
| **Thrust** | Forward (generated  by engines) | Drag | The force that pushes the aircraft forward through the air. |
| **Drag** | Backward (air resistance) | Thrust | The resistance or friction the air exerts on the moving aircraft. |

**In stable  flight:**
* **Lift = Weight**
* **Thrust = Drag**

To take off, the pilot must generate **Lift

In [4]:
# Tools - Define karna

from langchain.tools import tool

@tool
def get_weather(location: str) -> str:
  """Get the weather in a given location"""
  return f"The weather in {location} is sunny"

# Tool ko model ke saath bind karna
model_with_tools = model.bind_tools([get_weather])

# Model ko invoke karna
response = model_with_tools.invoke("What is the weather in Berlin?")
print(response)

if response.tool_calls:
    tool_call = response.tool_calls[0]
    print(f"\nTool Name: {tool_call['name']}")
    print(f"Arguments: {tool_call['args']}")
    print(f"Tool Call ID: {tool_call['id']}")
    
    # Actual tool ko execute karna
    location = tool_call['args']['location']
    result = get_weather.invoke({"location": location})
    print(f"\nTool Result: {result}")

content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 52, 'total_tokens': 66, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-Cx9CMM8VaMf1fhxRMJikA2IfgSzpD', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019bb1b1-47a3-76d2-852f-f1becb6337a6-0' tool_calls=[{'name': 'get_weather', 'args': {'location': 'Berlin'}, 'id': 'call_O8gCCu3eEQzSP6kUIQe7pcq4', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 52, 'output_tokens': 14, 'total_tokens': 66, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}

Tool Name: get_weather
Arg

In [11]:
# Tool calls ko dekhna
print("Tool Calls:", response.tool_calls)

# Tool call details
if response.tool_calls:
    tool_call = response.tool_calls[0]
    print(f"\nTool Name: {tool_call['name']}")
    print(f"Arguments: {tool_call['args']}")
    print(f"Tool Call ID: {tool_call['id']}")
    
    # Actual tool ko execute karna
    location = tool_call['args']['location']
    result = get_weather.invoke({"location": location})
    print(f"\nTool Result: {result}")

Tool Calls: [{'name': 'get_weather', 'args': {'location': 'Berlin'}, 'id': 'call_u7xrnlugjuIs99qltiW3LeYD', 'type': 'tool_call'}]

Tool Name: get_weather
Arguments: {'location': 'Berlin'}
Tool Call ID: call_u7xrnlugjuIs99qltiW3LeYD

Tool Result: The weather in Berlin is sunny


### Multiple Tools Example

In [10]:
# Multiple tools define karna
from langchain.tools import tool

@tool
def get_temperature(location: str) -> str:
    """Get the temperature in a given location in Celsius"""
    # Dummy data - real mein API call hogi
    temps = {"Berlin": "15", "Paris": "18", "London": "12", "Tokyo": "20"}
    return f"The temperature in {location} is {temps.get(location, '25')}°C"

@tool
def get_humidity(location: str) -> str:
    """Get the humidity level in a given location"""
    return f"The humidity in {location} is 65%"

@tool  
def calculate_sum(a: int, b: int) -> int:
    """Add two numbers together"""
    return a + b

# Sabhi tools ko bind karna
model_with_multiple_tools = model.bind_tools([get_weather, get_temperature, get_humidity, calculate_sum])

# Test karna
response = model_with_multiple_tools.invoke("What is the temperature and humidity in Berlin?")
print("Response:", response)
print("\nTool Calls:", response.tool_calls)

if response.tool_calls:
    tool_call = response.tool_calls[0]
    print(f"\nTool Name: {tool_call['name']}")
    print(f"Arguments: {tool_call['args']}")
    print(f"Tool Call ID: {tool_call['id']}")
    
    # Actual tool ko execute karna
    location = tool_call['args']['location']
    result = get_humidity.invoke({"location": location})
    print(f"\nTool Result: {result}")

Response: content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 125, 'total_tokens': 170, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-Cx9F8vn365Z4MWGc0LQP4ST0Zngkm', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019bb1b3-ede2-7c90-920b-f59ac952579c-0' tool_calls=[{'name': 'get_temperature', 'args': {'location': 'Berlin'}, 'id': 'call_PCKXoAJJLcnDvfvlvK7WTrLG', 'type': 'tool_call'}, {'name': 'get_humidity', 'args': {'location': 'Berlin'}, 'id': 'call_0s3QpZDOzCxAFBmIPHzdKLzb', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 125, 'output_tokens': 45, 'total_tokens': 170,

### Complete Tool Execution Flow

In [13]:
# Tool execution helper function
def execute_tool_calls(response, tools_dict):
    """
    Tool calls ko execute karta hai aur results return karta hai
    """
    if not response.tool_calls:
        return response.content
    
    results = []
    for tool_call in response.tool_calls:
        tool_name = tool_call['name']
        tool_args = tool_call['args']
        
        print(f"\n🔧 Executing Tool: {tool_name}")
        print(f"📝 Arguments: {tool_args}")
        
        # Tool ko execute karna
        if tool_name in tools_dict:
            result = tools_dict[tool_name].invoke(tool_args)
            print(f"✅ Result: {result}")
            results.append({
                'tool': tool_name,
                'args': tool_args,
                'result': result
            })
        else:
            print(f"❌ Tool not found: {tool_name}")
    
    return results

# Tools dictionary banana
tools_dict = {
    'get_weather': get_weather,
    'get_temperature': get_temperature,
    'get_humidity': get_humidity,
    'calculate_sum': calculate_sum
}

# Example 1: Weather query
print("="*50)
print("Example 1: Weather Query")
print("="*50)
response1 = model_with_multiple_tools.invoke("What's the weather in Berlin?")
results1 = execute_tool_calls(response1, tools_dict)

# Example 2: Multiple tools
print("\n" + "="*50)
print("Example 2: Multiple Tools")
print("="*50)
response2 = model_with_multiple_tools.invoke("What is the temperature in Paris and calculate 25 + 37")
results2 = execute_tool_calls(response2, tools_dict)

Example 1: Weather Query

🔧 Executing Tool: get_weather
📝 Arguments: {'location': 'Berlin'}
✅ Result: The weather in Berlin is sunny

Example 2: Multiple Tools

🔧 Executing Tool: get_temperature
📝 Arguments: {'location': 'Paris'}
✅ Result: The temperature in Paris is 18°C

🔧 Executing Tool: calculate_sum
📝 Arguments: {'a': 25, 'b': 37}
✅ Result: 62


### Key Concepts Summary

**Tools kya hain?**
- Functions jo model call kar sakta hai
- External data fetch karne ke liye (APIs, databases, etc.)
- Calculations perform karne ke liye
- Real-world actions execute karne ke liye

**Tool Definition:**
```python
@tool
def function_name(param: type) -> return_type:
    """Description for the model"""
    return result
```

**Important Points:**
1. **Docstring zaruri hai** - Model isko padhkar decide karta hai kab tool use karna hai
2. **Type hints** - Parameters aur return type clear hone chahiye
3. **bind_tools()** - Model ko tools attach karta hai
4. **tool_calls** - Model ki request for tool execution
5. **invoke()** - Actual tool execution

**Flow:**
1. User query → Model
2. Model decides tool chahiye
3. Model returns tool_calls (not actual result)
4. Hum manually tool execute karte hain
5. Result ko model ko wapas bhej sakte hain for final response